In [1]:
#import evidently

In [2]:
# from evidently.dashboard import Dashboard
# from evidently.tabs import DataDriftTab, CatTargetDriftTab
# from evidently.profile_sections import DataDriftProfileSection

C:\Users\ZagkouI\AppData\Roaming\Python\Python36\site-packages\evidently\analyzers\__init__.py:3: UserWarning: analyzers are deprecated, use metrics instead
  warnings.warn("analyzers are deprecated, use metrics instead")
C:\Users\ZagkouI\AppData\Roaming\Python\Python36\site-packages\evidently\dashboard\__init__.py:8: UserWarning:

dashboards are deprecated, use metrics instead

C:\Users\ZagkouI\AppData\Roaming\Python\Python36\site-packages\evidently\tabs\__init__.py:9: UserWarning:

'import evidently.tabs' is deprecated, use 'import evidently.dashboard.tabs'

C:\Users\ZagkouI\AppData\Roaming\Python\Python36\site-packages\evidently\model_profile\__init__.py:8: UserWarning:

model profiles are deprecated, use metrics instead

C:\Users\ZagkouI\AppData\Roaming\Python\Python36\site-packages\evidently\profile_sections\__init__.py:9: UserWarning:

'import evidently.profile_sections' is deprecated, use 'import evidently.model_profile.sections'



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import re

In [8]:
#from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
#from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV
#from sklearn.externals import joblib

In [23]:
import shap

ModuleNotFoundError: No module named 'numba.core'

In [24]:
pd.set_option('display.max_columns', 550)
pd.set_option('display.max_rows', 550)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# LOAD TRAINING SET

In [10]:
previous_dataset = pd.read_csv('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/final_data/train/final_customer_db.csv', index_col=0)

In [11]:
previous_dataset.head()

,MSISDN,M1_TTL_OUT_CALLS,M1_TTL_OUT_MINUTES,M1_TTL_OUT_REVENUE,M1_V_CALLS_TO_FIXED_LINES,M1_V_MINUTES_TO_FIXED_LINES,M1_V_REVENUE_TO_FIXED_LINES,M1_V_CALLS_TO_COMPETITION,M1_V_MINUTES_TO_COMPETITION,M1_V_REVENUE_TO_COMPETITION,M1_V_CALLS_TO_INTERNATIONAL,M1_V_MINUTES_TO_INTERNATIONAL,M1_V_REVENUE_TO_INTERNATIONAL,M1_GPRS_SESSION,M1_GPRS_REVENUE,M1_GPRS_VOLUME,M1_RECHARGES_NUMBER,M1_RECHARGES_VALUE,M1_OUT_DAYS,M1_INC_DAYS,M1_BUNDLE_REVENUE,M2_TTL_OUT_CALLS,M2_TTL_OUT_MINUTES,M2_TTL_OUT_REVENUE,M2_V_CALLS_TO_FIXED_LINES,M2_V_MINUTES_TO_FIXED_LINES,M2_V_REVENUE_TO_FIXED_LINES,M2_V_CALLS_TO_COMPETITION,M2_V_MINUTES_TO_COMPETITION,M2_V_REVENUE_TO_COMPETITION,M2_V_CALLS_TO_INTERNATIONAL,M2_V_MINUTES_TO_INTERNATIONAL,M2_V_REVENUE_TO_INTERNATIONAL,M2_GPRS_SESSION,M2_GPRS_REVENUE,M2_GPRS_VOLUME,M2_RECHARGES_NUMBER,M2_RECHARGES_VALUE,M2_OUT_DAYS,M2_INC_DAYS,M2_BUNDLE_REVENUE,M3_TTL_OUT_CALLS,M3_TTL_OUT_MINUTES,M3_TTL_OUT_REVENUE,M3_V_CALLS_TO_FIXED_LINES,M3_V_MINUTES_TO_FIXED_LINES,M3_V_REVENUE_TO_FIXED_LINES,M3_V_CALLS_TO_COMPETITION,M3_V_MINUTES_TO_COMPETITION,M3_V_REVENUE_TO_COMPETITION,M3_V_CALLS_TO_INTERNATIONAL,M3_V_MINUTES_TO_INTERNATIONAL,M3_V_REVENUE_TO_INTERNATIONAL,M3_GPRS_SESSION,M3_GPRS_REVENUE,M3_GPRS_VOLUME,M3_RECHARGES_NUMBER,M3_RECHARGES_VALUE,M3_OUT_DAYS,M3_INC_DAYS,M3_BUNDLE_REVENUE,M4_TTL_OUT_CALLS,M4_TTL_OUT_MINUTES,M4_TTL_OUT_REVENUE,M4_V_CALLS_TO_FIXED_LINES,M4_V_MINUTES_TO_FIXED_LINES,M4_V_REVENUE_TO_FIXED_LINES,M4_V_CALLS_TO_COMPETITION,M4_V_MINUTES_TO_COMPETITION,M4_V_REVENUE_TO_COMPETITION,M4_V_CALLS_TO_INTERNATIONAL,M4_V_MINUTES_TO_INTERNATIONAL,M4_V_REVENUE_TO_INTERNATIONAL,M4_GPRS_SESSION,M4_GPRS_REVENUE,M4_GPRS_VOLUME,M4_RECHARGES_NUMBER,M4_RECHARGES_VALUE,M4_OUT_DAYS,M4_INC_DAYS,M4_BUNDLE_REVENUE,M5_TTL_OUT_CALLS,M5_TTL_OUT_MINUTES,M5_TTL_OUT_REVENUE,M5_V_CALLS_TO_FIXED_LINES,M5_V_MINUTES_TO_FIXED_LINES,M5_V_REVENUE_TO_FIXED_LINES,M5_V_CALLS_TO_COMPETITION,M5_V_MINUTES_TO_COMPETITION,M5_V_REVENUE_TO_COMPETITION,M5_V_CALLS_TO_INTERNATIONAL,M5_V_MINUTES_TO_INTERNATIONAL,M5_V_REVENUE_TO_INTERNATIONAL,M5_GPRS_SESSION,M5_GPRS_REVENUE,M5_RECHARGES_NUMBER,M5_RECHARGES_VALUE,M5_GPRS_VOLUME,M5_OUT_DAYS,M5_INC_DAYS,M5_BUNDLE_REVENUE,M6_TTL_OUT_CALLS,M6_TTL_OUT_MINUTES,M6_TTL_OUT_REVENUE,M6_V_CALLS_TO_FIXED_LINES,M6_V_MINUTES_TO_FIXED_LINES,M6_V_REVENUE_TO_FIXED_LINES,M6_V_CALLS_TO_COMPETITION,M6_V_MINUTES_TO_COMPETITION,M6_V_REVENUE_TO_COMPETITION,M6_V_CALLS_TO_INTERNATIONAL,M6_V_MINUTES_TO_INTERNATIONAL,M6_V_REVENUE_TO_INTERNATIONAL,M6_GPRS_SESSION,M6_GPRS_REVENUE,M6_GPRS_VOLUME,M6_RECHARGES_NUMBER,M6_RECHARGES_VALUE,M6_OUT_DAYS,M6_INC_DAYS,M6_BUNDLE_REVENUE,TARIFF_PLAN,CONNECTION_DAY,SMARTPHONE_FLAG,INSERTED,M1_MINUTES_PER_CALL,M2_MINUTES_PER_CALL,M3_MINUTES_PER_CALL,M4_MINUTES_PER_CALL,M5_MINUTES_PER_CALL,M6_MINUTES_PER_CALL,TENURE_MONTHS,CUSTOMER_TENURE,M1_ARPU,M2_ARPU,M3_ARPU,M4_ARPU,M5_ARPU,M6_ARPU,M1_ROC_TTL_OUT_CALLS,M1_ROC_TTL_OUT_MINUTES,M1_ROC_TTL_OUT_REVENUE,M1_ROC_V_CALLS_TO_FIXED_LINES,M1_ROC_V_MINUTES_TO_FIXED_LINES,M1_ROC_V_REVENUE_TO_FIXED_LINES,M1_ROC_V_CALLS_TO_COMPETITION,M1_ROC_V_MINUTES_TO_COMPETITION,M1_ROC_V_REVENUE_TO_COMPETITION,M1_ROC_V_CALLS_TO_INTERNATIONAL,M1_ROC_V_MINUTES_TO_INTERNATIONAL,M1_ROC_V_REVENUE_TO_INTERNATIONAL,M1_ROC_GPRS_SESSION,M1_ROC_GPRS_REVENUE,M1_ROC_GPRS_VOLUME,M1_ROC_RECHARGES_NUMBER,M1_ROC_RECHARGES_VALUE,M1_ROC_OUT_DAYS,M1_ROC_INC_DAYS,M1_ROC_BUNDLE_REVENUE,M1_ROC_MINUTES_PER_CALL,M1_ROC_ARPU,M13_AVG_TTL_OUT_CALLS,M46_AVG_TTL_OUT_CALLS,M16_AVG_TTL_OUT_CALLS,M13_AVG_TTL_OUT_MINUTES,M46_AVG_TTL_OUT_MINUTES,M16_AVG_TTL_OUT_MINUTES,M13_AVG_TTL_OUT_REVENUE,M46_AVG_TTL_OUT_REVENUE,M16_AVG_TTL_OUT_REVENUE,M13_AVG_V_CALLS_TO_FIXED_LINES,M46_AVG_V_CALLS_TO_FIXED_LINES,M16_AVG_V_CALLS_TO_FIXED_LINES,M13_AVG_V_MINUTES_TO_FIXED_LINES,M46_AVG_V_MINUTES_TO_FIXED_LINES,M16_AVG_V_MINUTES_TO_FIXED_LINES,M13_AVG_V_REVENUE_TO_FIXED_LINES,M46_AVG_V_REVENUE_TO_FIXED_LINES,M16_AVG_V_REVENUE_TO_FIXED_LINES,M13_AVG_V_CALLS_TO_COMPETITION,M46_AVG_V_CALLS_TO_COMPETITION,M16_AVG_V_CALLS_TO

In [5]:
out_dir = 'C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/models/'

In [6]:
previous_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 907206 entries, 0 to 907205
Columns: 265 entries, MSISDN to HIGHER_BUNDLE
dtypes: float64(244), int64(10), object(11)
memory usage: 1.8+ GB


In [7]:
previous_dataset["HIGHER_BUNDLE"].value_counts()

0.0    868096
1.0     39110
Name: HIGHER_BUNDLE, dtype: int64

# FUNCTIONS

In [8]:
def downsampling(dataset, n_majority):
    dataset_major = dataset[(dataset['HIGHER_BUNDLE'] == 0)]
    dataset_minor = dataset[(dataset['HIGHER_BUNDLE'] == 1)]
    major = dataset_major.sample(n=n_majority, random_state=10) #downsampling
    df_sampled = pd.concat([major, dataset_minor])
    df_shuffled = df_sampled.sample(frac=1)
    
    return df_shuffled

In [9]:
def split_x_y(df):
    X = df.drop(["MSISDN","HIGHER_BUNDLE", "INSERTED", "CONNECTION_DAY", "Periferiaki_enotita", "Name", "POST_CODE"], axis=1)
    y = df["HIGHER_BUNDLE"].values
    
    return X,y

In [10]:
def encoding_data(X, y):
    le = LabelEncoder()
    objects = X.select_dtypes(include=['object'])
    object_names = list(objects.columns.values)
    
    # Encoding all the categorical columns
    for object_name in object_names:
        if (X[object_name].nunique() > 2):
            enc_pc = pd.get_dummies(X[object_name], drop_first = True)
            X = X.drop(object_name, axis = 1)
            X = pd.concat([X,enc_pc], axis = 1)
        elif (X[object_name].nunique() <= 2):
            le.fit(X[object_name].astype(str))
            X[object_name] = le.transform(X[object_name].astype(str))


    y = le.fit_transform(y)
    
    return X,y

In [1]:
def estimate_proportion(y):
    count_train_0= (y == 0).sum()
    count_train_1= (y == 1).sum()
    
    proportion = float(float(count_train_0) / float(count_train_1))
    
    return proportion

In [12]:
def split_train_test(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)
    
    return X_train, X_test, y_train, y_test

In [13]:
def evaluation(X_test, y_test, classifier):
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    target_names = ['No higher bundle', 'Higher Bundle']
    evaluation_report = classification_report(y_test, y_pred, target_names=target_names)
    
    return cm, accuracy, evaluation_report

In [ ]:
def feature_importance(X_test, classifier):
    explainer = shap.TreeExplainer(classifier)
    shap_values = explainer.shap_values(X_test)
    shap.summary_plot(shap_values, X_test, feature_names, plot_type="bar")
    
    shap.summary_plot(shap_values, X_test, feature_names)

# PREPARE DATASET

In [14]:
# Downsampling
previous_sampled_dataset = downsampling(previous_dataset, n_majority= 100000)  # change this number
# Define X and y
X_previous, y_previous = split_x_y(previous_sampled_dataset)
# Encoding
X_previous, y_previous = encoding_data(X_previous,y_previous)
# Save feature name
feature_names= np.array(X_previous.columns)

In [15]:
previous_proportion = estimate_proportion(y_previous)
print(previous_proportion)

2.0


In [16]:
X_previous_new = X_previous.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# TRAINING

In [17]:
lgbm = lgb.LGBMClassifier(boosting_type = 'gbdt',
                                    importance_type = 'gain',
                                    scale_pos_weight= previous_proportion,
                                    n_estimators = 1000,
                                    learning_rate=0.01, metric='auc', silent=True, n_jobs=-1, random_state=10) 

In [23]:
# lgbm = lgb.LGBMClassifier(boosting_type = 'gbdt',
#                                     importance_type = 'gain',
#                                     scale_pos_weight= 12,
#                                     n_estimators = 500,
#                                     learning_rate=0.1, metric='auc', silent=True, n_jobs=-1, random_state=10) 

In [18]:
lgbm.fit(X_previous_new, y_previous)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='gain', learning_rate=0.01, max_depth=-1,
        metric='auc', min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=1000, n_jobs=-1, num_leaves=31,
        objective=None, random_state=10, reg_alpha=0.0, reg_lambda=0.0,
        scale_pos_weight=2.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

In [25]:
#joblib.dump(lgbm, os.path.join(out_dir, 'model.pkl'))

['C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/models/model.pkl']

# EVALUATION

In [19]:
next_dataset = pd.read_csv('C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/final_data/test/final_customer_db.csv', index_col=0)

In [20]:
# Define X and y
X_next, y_next = split_x_y(next_dataset)
# Encoding
X_next, y_next = encoding_data(X_next,y_next)
# Save feature name
feature_names = np.array(X_next.columns)

In [21]:
cm, acc, class_report = evaluation(X_next, y_next, lgbm)
print(cm)
print(acc)
print(class_report)

C:\Users\zagkoui\AppData\Local\conda\conda\envs\irida\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[[673284 202319]
 [  8651  23578]]
0.7676111879731051
                  precision    recall  f1-score   support

No higher bundle       0.99      0.77      0.86    875603
   Higher Bundle       0.10      0.73      0.18     32229

     avg / total       0.96      0.77      0.84    907832



# Feature importance

In [ ]:
feature_importance(X_next, loaded_model)

In [ ]:
pred_y_propab = loaded_model.predict_proba(X_next)
pred_y_propab[:,1]
condition = pred_y_propab[:,1] >= 0.5
len(pred_y_propab[:,1][condition])

# TUNING

In [23]:
for booster in ['gbdt', 'dart']:
    for scale in [2, 4]:
    #for scale in [feb_proportion/2, feb_proportion]:
        for est in [100, 500, 1000]:
            for lr in [0.01, 0.1]:
                print (booster, scale, est, lr)
                # ----------------TRAINING-------------------
                lgbm = lgb.LGBMClassifier(boosting_type = booster,
                                    importance_type = 'gain',
                                    scale_pos_weight= scale,
                                    n_estimators = est,
                                    learning_rate=lr, metric='auc', silent=True, n_jobs=-1, random_state=10) 

                lgbm.fit(X_previous_new, y_previous)
                # ----------------TESTING-------------------
                cm, acc, class_report = evaluation(X_next, y_next, lgbm)
                print(cm)
                print(acc)
                print(class_report)

                # -----------------EVALUATION-------------------
                pred_y_propab = lgbm.predict_proba(X_next)
                pred_y_propab[:,1]
                condition = pred_y_propab[:,1] >= 0.5
                len(pred_y_propab[:,1][condition])

('gbdt', 2, 100, 0.01)


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='gain', learning_rate=0.01, max_depth=-1,
        metric='auc', min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
        objective=None, random_state=10, reg_alpha=0.0, reg_lambda=0.0,
        scale_pos_weight=2, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

C:\Users\zagkoui\AppData\Local\conda\conda\envs\irida\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[[816749  58854]
 [ 23284   8945]]
0.9095229073220595
                  precision    recall  f1-score   support

No higher bundle       0.97      0.93      0.95    875603
   Higher Bundle       0.13      0.28      0.18     32229

     avg / total       0.94      0.91      0.92    907832



array([0.10791508, 0.43173956, 0.14413622, ..., 0.26971628, 0.43461874,
       0.24286671])

...

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
        importance_type='gain', learning_rate=0.1, max_depth=-1,
        metric='auc', min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=1000, n_jobs=-1, num_leaves=31,
        objective=None, random_state=10, reg_alpha=0.0, reg_lambda=0.0,
        scale_pos_weight=8, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

C:\Users\zagkoui\AppData\Local\conda\conda\envs\irida\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[[486624 388979]
 [  2133  30096]]
0.5691802007419875
                  precision    recall  f1-score   support

No higher bundle       1.00      0.56      0.71    875603
   Higher Bundle       0.07      0.93      0.13     32229

     avg / total       0.96      0.57      0.69    907832



array([0.00101694, 0.73737867, 0.00082597, ..., 0.37711577, 0.69291584,
       0.60177926])

419075